# Treci domaci zadatak - Katarina Petrovic 2021/0068

In [ ]:
from skimage import io
from skimage import util
from skimage import filters
from skimage import transform
from skimage import color

import numpy as np

from scipy import ndimage

import matplotlib.pyplot as plt

import math

## Prvi zadatak - canny_edge_detection

In [ ]:
def canny_edge_detection(img_in, sigma, thr_low, thr_high):
    #Dimenzije ulazne slike, M - broj vrsta, N - broj kolona
    [M, N] = np.shape(img_in)

    #Filtriranje Gausovim filtrom - suzbijamo sum
    img_gauss = filters.gaussian(img_in, sigma, truncate=3, mode='nearest')
    
    #Maske Sobelovog filtra
    Hx = np.array([[-1, -2, -1],[0, 0, 0],[1, 2, 1]])
    Hy = np.transpose(Hx)
    
    #Primenjujemo Sobelov filtar na zasumljenu sliku - dobijamo gradijente po x i y osi
    Gx = ndimage.correlate(img_gauss, Hx, mode='nearest')
    Gy = ndimage.correlate(img_gauss, Hy, mode='nearest')
    
    #Magnituda i ugao gradijenta
    mag = np.sqrt(np.square(Gx) + np.square(Gy))
    angle = np.degrees(np.arctan2(Gy, Gx))
    
    #Matrica slabih ivica, jakih ivica i neivicnih piksela
    weak_ids = np.zeros_like(img_in) 
    strong_ids = np.zeros_like(img_in)               
    no_edge_ids = np.zeros_like(img_in)
    
    #Potiskivanje lokalnih nemaksimuma za svaki piksel i definisanje slabih i jakih ivica
    for i in range(M): 
        for j in range(N): 
            grad_ang = angle[i, j] 
            
            #Zavisno od ugla gradijenta definisemo koje piksele zelimo da razmatramo kao susede
            if (grad_ang<= 22.5 and grad_ang>-22.5) or grad_ang<= -157.5 or grad_ang>157.5: 
                #Gornji i donji sused
                neighb_1_i, neighb_1_j = i-1, j 
                neighb_2_i, neighb_2_j = i + 1, j
              
            elif (grad_ang>22.5 and grad_ang<=67.5) or (grad_ang>-157.5 and grad_ang<=-112.5): 
                #Dijagonalni susedi
                neighb_1_i, neighb_1_j = i-1, j-1
                neighb_2_i, neighb_2_j = i + 1, j + 1
              

            elif (grad_ang>67.5 and grad_ang<=112.5) or (grad_ang>-112.5 and grad_ang<=-67.5): 
                #Levi i desni sused
                neighb_1_i, neighb_1_j = i, j-1
                neighb_2_i, neighb_2_j = i, j + 1
              

            elif (grad_ang>112.5 and grad_ang<=157.5) or (grad_ang>-67.7 and grad_ang<=-22.5): 
                #Dijagonalni susedi
                neighb_1_i, neighb_1_j = i-1, j + 1
                neighb_2_i, neighb_2_j = i + 1, j-1
    
            #Proveravamo da li je piksel "jaca ivica" od suseda -> ako nije, postavljamo njegovu vrednost na 0
            if M>neighb_1_i>= 0 and N>neighb_1_j>= 0: 
                if mag[i, j]<mag[neighb_1_i, neighb_1_j]: 
                    mag[i, j]= 0
   
            if M>neighb_2_i>= 0 and N>neighb_2_j>= 0: 
                if mag[i, j]<mag[neighb_2_i, neighb_2_j]: 
                    mag[i, j]= 0
            
            #Piksel svrstavamo u jaku ili slabu ivicu ili u neivične piksele
            if mag[i, j] > thr_high:
                strong_ids[i, j] = 1
            elif mag[i,j] >= thr_low and mag[i, j] <= thr_high:
                weak_ids[i, j] = 1
            else:
                no_edge_ids[i, j] = 1

    
    #Dodajemo slabe ivice koje su povezane sa jakim ivicama u jake ivice
    changes = True
    
    #Sve dok ne udjemo u stacionarno stanje
    while changes:
        changes = False
        
        weak_indices = np.argwhere(weak_ids)
        for i, j in weak_indices:
            #Posmatrmo okolinu slabo ivicnog piksela i u njoj trazimo jaku ivicu
            #Ako ona postoji prebacujemo piksel u jake ivice
            if np.any(strong_ids[max(0, i - 1):min(M, i + 2), max(0, j - 1):min(N, j + 2)]):
                strong_ids[i, j] = 1
                weak_ids[i, j] = 0
                changes = True
   
    
    img_edges = strong_ids
    
    return img_edges


## Testiranje funkcije canny_edge_detection

In [ ]:
img = util.img_as_float(io.imread('../sekvence/van.tif'))
#img = color.rgb2gray(img)
img_edges = canny_edge_detection(img, 0.7, 0.35, 0.55)

fix, axes = plt.subplots(1, 1, figsize=(20, 8))
axes.imshow(img, cmap='gray')
axes.set_title('Ulazna slika')
axes.axis('off')

fix, axes = plt.subplots(1, 1, figsize=(20, 8))
axes.imshow(img_edges, cmap='gray')
axes.set_title('Izlazna slika (sigma = 0.7, thr_low = 0.35, thr_high = 0.55)')
axes.axis('off')


## Drugi zadatak - get_line_segments

In [ ]:
def get_line_segments(img_edges, line, min_length, max_gap, tolerancy):
    
    #Ekstrakcija ulaznih parametara - theta je ugao, rho je najkrace rastojanje duzi od koord pocetka
    theta, rho = line
    
    #Specificni slucajevi i obrada ulaznih parametara
    if theta < -np.pi/2 or theta > np.pi/2:
        print('Greska: Ugao mora biti u opsegu [-pi/2, pi/2]')
        return (0, [], [])
    if min_length <= 0:
        print('Greska: Minimalna duzina segmenta mora biti pozitivna vrednost')
        return (0, [], [])
    if max_gap < 0:
        print('Greska: Maksimalna rupa u segmentu mora biti pozitivna vrednost')
        return (0, [], [])
    if tolerancy <= 0:
        print('Greska: Tolerancija mora biti pozitivna vrednost')
        return (0, [], [])
    
    
    
    #Segment pot su svi pikseli koji su potencijalno segmenti - nalaze se na pravcu (sa tolerancijom) - za vizuelni prikaz
    segment_pot = np.zeros_like(img_edges)
    
    #Segment pot arr su svi ti pikseli samo uzete njihove (i, j) koordinate kako bi kasnije lakse manipulisali podacima
    segment_pot_arr = []
    M, N = img_edges.shape
    #Ubacujemo piksele iz slike img_edges koji se nalaze na pravcu (sa tolerancijom)
    for i in range(M):
        for j in range(N):
            
            #Distance je rastojanje od zadatog pravca
            distance = abs(np.round(i * np.sin(theta) + j * np.cos(theta)) - rho)
            
            #Ako je distanca manja od 1 - na pravcu smo 
            if distance < 1:
                #Provera da li je piksel ivicni - ako jeste stavimo da pripada potencijalnom segmentu
                if img_edges[i,j] == 1:
                    segment_pot[i, j] = 1
                    segment_pot_arr.append([i, j])
                #Ako nije ivicni, proverimo da li u njegovoj tolerancy okolini postoji ivicni
                #Ako da, stavimo da pripada potencijalnom segmentu
                else:
                    i_min = max(0, i - tolerancy)
                    i_max = min(M, i + tolerancy + 1)
                    j_min = max(0, j - tolerancy)
                    j_max = min(N, j + tolerancy + 1)
                    window = img_edges[i_min:i_max, j_min:j_max]
                    if np.any(window) == True:
                        segment_pot[i,j] = 1
                        segment_pot_arr.append([i, j])
    
    #Ako nemamo nijedan piksel koji potencijalno pripada segmentu, vracamo (0, [], [])
    if not segment_pot_arr:
        return (0, [], [])
    #Ako imamo piksele koji ce potencijalno pripadati pikselu, inicijalizujemo segment_coords
    #Prvi (potencijalni segment) ce poceti prvim pikselom iz segment_pot_arr
    else:
        segment_coords = [segment_pot_arr[0][0], segment_pot_arr[0][1]]
    
    #Vrednosti max_gap i min_length skalirane na x i y osu zavisno od ugla theta
    #Koristimo abs za sin/cos da ne bi razmisljali o znakovima, bitno je samo rastojanje piksela
    max_gap_x = max_gap*abs(np.cos(theta))
    max_gap_y = max_gap*abs(np.sin(theta))
    min_length_x = min_length*abs(np.cos(theta))
    min_length_y = min_length*abs(np.sin(theta))
    
    
    #Dodatak za sin/cos priblizno jednak 0 
    if min_length_x < 1:
        min_length_x = 0
    if min_length_y < 1:
        min_length_y = 0
    if max_gap_x < 1:
        max_gap_x = 0
    if max_gap_y < 1:
        max_gap_y = 0
    
    #Piksele koji su na vecem rastojanju od max_gap uzimamo kao da su delovi razlicitih segmenata
    #Pamtimo koordinate pocetka i kraja svakog (i za sada potencijalnog) segmenta
    #Trenutno ne uzimamo u obzir min_length, kasnije cemo izbacivati segmente koji su manji od min_length
    for elem in range(0,len(segment_pot_arr)-1):
        #(i,j) koordinate trenutnog piksela i narednog
        i_coord = segment_pot_arr[elem][0]
        j_coord = segment_pot_arr[elem][1]
        i_next = segment_pot_arr[elem+1][0]
        j_next = segment_pot_arr[elem+1][1]
    
        #Rastojanje trenutnog piksela i narednog
        dist_x = abs(i_next-i_coord)
        dist_y = abs(j_next-j_coord)
        
        #U slucaju da je neki sin ili cos = 0 zelimo da proveravamo rastojanje samo po horizontali/vertikali
        if max_gap_x == 0:
            if dist_y > max_gap_y:
                segment_coords.append(i_coord)
                segment_coords.append(j_coord)
                segment_coords.append(i_next)
                segment_coords.append(j_next)
                
        elif max_gap_y == 0:
            if dist_x > max_gap_x:
                segment_coords.append(i_coord)
                segment_coords.append(j_coord)
                segment_coords.append(i_next)
                segment_coords.append(j_next)
                
        #Ako ugao nije nula, proveramo da li je naredni piksel dovoljno daleko po oba pravca
        #Ako da, dodajemo koordinate trenutnog piksela u segment_coords
        #To ce biti kraj trenutnog segmenta i koordinate narednog piksela - to ce biti pocetak narednog piksela
        #Ako ne, ne dodajemo ga - on je deo segmenta
        elif dist_x > max_gap_x or dist_y > max_gap_y:
            segment_coords.append(i_coord)
            segment_coords.append(j_coord)
            segment_coords.append(i_next)
            segment_coords.append(j_next)
            
    #Dodajemo poslednji piksel iz potencijalnih segmenata - on predstavlja kraj poslednjeg segmenta
    segment_coords.append(segment_pot_arr[-1][0])
    segment_coords.append(segment_pot_arr[-1][1])
    #Segment_coords je niz od 4*(broj_segmenata) za sad - za svaki segment imamo pocetne i krajnje koordinate  

    #Segment_coords_copy ce biti lista iz koje su izbaceni elementi koji su manji od zadate duzine
    segment_coords_copy = []
    #Pamtimo velicine segmenata ukoliko su one vece od min_length
    segments_size = []
    
    #Proveravamo duzine segmenata
    for i in range(0, len(segment_coords), 4):
        #Ako je duzina veca od zadate, u segment_coords_copy dodajemo koordinate segmenta i pamtimo duzinu segmenta u segments_size
        if abs(segment_coords[i]-segment_coords[i+2]) >= min_length_x and abs(segment_coords[i+1]-segment_coords[i+3]) >= min_length_y:
            segment_coords_copy.append(segment_coords[i])
            segment_coords_copy.append(segment_coords[i+1])
            segment_coords_copy.append(segment_coords[i+2])
            segment_coords_copy.append(segment_coords[i+3])
            
            seg_len = np.sqrt((segment_coords[i]-segment_coords[i+2])**2 + (segment_coords[i+1]-segment_coords[i+3])**2)
            segments_size.append(np.round(seg_len))
            
    segments_num = len(segments_size)
    segments_coord = []
    for i in range(0, len(segment_coords_copy), 4):
        segments_coord.append([(segment_coords_copy[i], segment_coords_copy[i + 1]), (segment_coords_copy[i + 2], segment_coords_copy[i + 3])])
    
    return (segments_num, segments_size, segments_coord)


## Testiranje funkcije get_line_segments

In [ ]:
img = util.img_as_float(io.imread('../sekvence/shapes_noisy.tif'))
img_edges = canny_edge_detection(img, 2.2, 0.17, 0.26)
    
fix, axes = plt.subplots(1, 1, figsize=(20, 8))
axes.imshow(img_edges, cmap='gray')
axes.set_title('Ivice slike')
axes.axis('off')

theta = np.pi/2
rho = 190
min_length = 25
max_gap = 5
tolerancy = 10

(segments_num, segments_size, segments_coord) = get_line_segments(img_edges, (theta, rho), min_length, max_gap, tolerancy)
print("Parametri:", "theta:",theta,", rho:",rho,", min_length:",min_length,", max_gap:",max_gap,", tolerancy:",tolerancy)
print("Broj segmenata:",segments_num)
print("Duzine segmenata:",segments_size)
print("Koordinate segmenata:",segments_coord)


## Treci zadatak - extract_time

### Pomocne funkcije hours i minutes, koriste se u funkciji extract_time

In [ ]:
def hours(theta, i_middle, i_center):
    if theta >= -np.pi/2 and theta < -np.pi/3:
        if i_middle < i_center:
            h = 9
        else:
            h = 3
    elif theta >= -np.pi/3 and theta < -np.pi/6:
        if i_middle < i_center:
            h = 10
        else:
            h = 4
    elif theta >= -np.pi/6 and theta < 0:
        if i_middle < i_center:
            h = 11
        else:
            h = 5
    elif theta >= 0 and theta < np.pi/6:
        if i_middle < i_center:
            h = 12
        else:
            h = 6
    elif theta >= np.pi/6 and theta < np.pi/3:
        if i_middle < i_center:
            h = 1
        else:
            h = 7
    elif theta > np.pi/3 and theta < np.pi/2:
        if i_middle < i_center:
            h = 2
        else:
            h = 8
    return h

In [ ]:
def minutes(theta, i_middle, i_center):
    if theta >= 0:
        if i_middle < i_center:
            m = round(theta/np.pi*2*15)
        else:
            m = round(theta/np.pi*2*15)+30
    elif theta < 0:
        if i_middle < i_center:
            m = round(15/np.pi*2*(theta+np.pi/2)+45)
        else:
            m = round(15/np.pi*2*(theta+np.pi/2)+15)
        
    return m

### Funkcija extract_time

In [ ]:
def extract_time(img_in):
    
    #Konvertujemo sliku u grayscale
    img_shape = img_in.shape
    M = img_shape[0]
    N = img_shape[1]
    
    if len(img_shape) == 2:
        img_gray = img_in
        img_hsv = img_in
    elif img_shape[2] == 3:
        img_gray = color.rgb2gray(img_in)
        img_hsv = color.rgb2hsv(img_in)
    elif img_shape[2] == 4:
        img_gray = color.rgb2gray(color.rgba2rgb(img_in))
        img_hsv = color.rgb2hsv(color.rgba2rgb(img_in))
    
    
    #Ako postoje crveni pikseli na slici pretvorimo ih u bele, to je crvena kazaljka
    for i in range(0,M):
        for j in range(0,N):
            if img_hsv[i,j,1] > 0.2 and img_hsv[i,j,2] > 0.2: 
                if img_hsv[i,j,0] > 0.85 or img_hsv[i,j,0] < 0.15:
                    img_gray[i,j] = 1
    
    #Primenjujemo realizovanu funkciju za detektovanje ivica
    img_edges = canny_edge_detection(img_gray, 0.8, 0.5, 0.6)
   
    #Hafova transformacija za linije - dobijamo pikove uglova i rastojanja
    tested_angles = np.linspace(-np.pi / 2, np.pi / 2, 100, endpoint=False)
    out, angles, d = transform.hough_line(img_edges, tested_angles)
    [intensities, peak_angles, peak_distances] = transform.hough_line_peaks(out, angles, d, min_distance=1, min_angle=10, threshold=np.amax(out)*0.3, num_peaks=10)
    
    #Centar sata
    i_center = round(M/2)
    j_center = round(N/2)
    
    #U dobijenim nizovima peak_angles i peak_distances se nalaze mnoge nezeljene vrednosti (npr. ponovljene slicne vrednosti)
    #Pravimo nove nizove peak_angles_real i peak_distances_real da bi ocitali vreme
    peak_angles_real = []
    peak_distances_real = []
    
    #Pamtimo i gde se nalazi sredina segmenta da bi znali da li je kazaljka u gornjoj ili donjoj polovini sata
    segment_middles = []
    segment_lengths = []
    
    for i in range(len(intensities)):
    
        #Zelimo samo prva tri max elementa da uzmemo - tri kazaljke (sati, minuti i sekunde ako postoje)
        if len(peak_angles_real) == 3:
            break

        theta = peak_angles[i]
        rho = peak_distances[i]
        
        #Primenjujemo realizovanu funkciju da bi detektovali da li postoje kazaljke na pravcima
        (segments_num, segments_size, segments_coord) = get_line_segments(img_edges, (theta, rho), 75, 7, 1)
        
        #Proveravamo da li je segment dovoljno dugacak i da li je dovoljno blizu centra sata
        far = False
        short = False
        
        if segments_num > 1:
            
            #Ako se desi da postoji vise od jednog segmenta koji je veci od zadate duzine
            max_seg = max(segments_size)
            seg_length = max_seg
            index = segments_size.index(max_seg)
            i_begin = segments_coord[index][0][0]
            j_begin = segments_coord[index][0][1]
            i_end = segments_coord[index][1][0]
            j_end = segments_coord[index][1][1]
            i_middle = (i_begin + i_end)/2
            
            
            if abs(i_middle-i_center) > min(M,N)/6:
                far = True
                
        elif segments_num == 1:
            seg_length = segments_size[0]
            i_begin = segments_coord[0][0][0]
            j_begin = segments_coord[0][0][1]
            i_end = segments_coord[0][1][0]
            j_end = segments_coord[0][1][1]
            i_middle = (i_begin + i_end)/2
            
            if abs(i_middle-i_center) > min(M,N)/6:
                far = True
        else:
            #Nemamo nijedan segment - svi segmenti su kratki da bi predstavljali kazaljku
            short = True

        #Prvo ubacimo najveci peak
        if len(peak_angles_real) == 0:
            if far == False and short == False:
                
                peak_angles_real.append(peak_angles[i])
                peak_distances_real.append(peak_distances[i])
                segment_middles.append(i_middle)
                segment_lengths.append(seg_length)
                
        #Zatim ostale koji su razliciti od vec ubacenih (sa malom tolerancijom jer je problematicno sa uglovima oko 0)
        else:
            exists = False
            for j in range(len(peak_angles_real)):
                if abs(peak_angles[i]-peak_angles_real[j])<0.15:
                    exists = True
            if exists == False and far == False and short == False:
                peak_angles_real.append(peak_angles[i])
                peak_distances_real.append(peak_distances[i])
                segment_middles.append(i_middle)
                segment_lengths.append(seg_length)

    #Zelimo da sortiramo opadajuce po duzinama segmenata
    sorting = sorted(zip(segment_lengths, peak_angles_real, peak_distances_real, segment_middles), key=lambda x: x[0], reverse=True)
        
    segment_lengths, peak_angles_real, peak_distances_real, segment_middles = zip(*sorting)

    
    #Na osnovu niza peak_angles_real i funkcija hours i minutes dobijamo vreme  
    if len(peak_angles_real) == 0:
        print('Greska: nema detektovanih kazaljki')
    if len(peak_angles_real) == 1:
        #Preklopljene kazaljke sata i minuta
        h = hours(peak_angles_real[0], segment_middles[0], i_center)
        m = minutes(peak_angles_real[0], segment_middles[0], i_center)
    if len(peak_angles_real) == 2:
        #Detektovali smo dve kazaljke - sate i minute
        #Veca je za minute
        h = hours(peak_angles_real[1], segment_middles[1], i_center)
        m = minutes(peak_angles_real[0], segment_middles[0], i_center)
    if len(peak_angles_real) == 3:
        #Detektovali smo 3 kazaljke, od kojih je jedna sekundara
        #Ovde treba neka provera koja od ovih je sekundara pa onda, za sad pretpostavimo:
        #Za sekunde je najveca, pa onda za minute, pa onda za sate
        h = hours(peak_angles_real[2], segment_middles[2], i_center)
        m = minutes(peak_angles_real[1], segment_middles[1], i_center)
    return [h,m]

## Testiranje funkcije extract_time

In [ ]:
img_in = util.img_as_float(io.imread('../sekvence/clocks/clock1.png'))

[h,m] = extract_time(img_in)

print("Vreme:", h, "h", m, "min")

fix, axes = plt.subplots(1, 1, figsize=(20, 8))
axes.imshow(img_in, cmap='gray')
axes.set_title('Ulazna slika')
axes.axis('off')

In [ ]:
#Svi primeri

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock1.png'))

[h,m] = extract_time(img_in)

print("Slika 1 - vreme:", h,"h", m, "min")

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock2.png'))

[h,m] = extract_time(img_in)

print("Slika 2 - vreme:", h,"h", m, "min")

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock3.png'))

[h,m] = extract_time(img_in)

print("Slika 3 - vreme:", h,"h", m, "min")

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock4.png'))

[h,m] = extract_time(img_in)

print("Slika 4 - vreme:", h,"h", m, "min")

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock5.png'))

[h,m] = extract_time(img_in)

print("Slika 5 - vreme:", h,"h", m, "min")

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock6.png'))

[h,m] = extract_time(img_in)

print("Slika 6 - vreme:", h,"h", m, "min")

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock7.jpg'))

[h,m] = extract_time(img_in)

print("Slika 7 - vreme:", h,"h", m, "min")

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock8.jpg'))

[h,m] = extract_time(img_in)

print("Slika 8 - vreme:", h,"h", m, "min")

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock9.jpg'))

[h,m] = extract_time(img_in)

print("Slika 9 - vreme:", h,"h", m, "min")

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock10.png'))

[h,m] = extract_time(img_in)

print("Slika 10 - vreme:", h,"h", m, "min")

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock11.png'))

[h,m] = extract_time(img_in)

print("Slika 11 - vreme:", h,"h", m, "min")

img_in = util.img_as_float(io.imread('../sekvence/clocks/clock12.jpg'))

[h,m] = extract_time(img_in)

print("Slika 12 - vreme:", h,"h", m, "min")
